## Baggin

In [3]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import statsmodels.api as sm
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV
from skopt.space import Real
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import roc_auc_score


In [4]:
defa=pd.read_csv("C:/Users/munoz/Downloads/lab_apre_est/Default.csv")
defa.head()

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138950
3,No,No,529.250605,35704.493940
4,No,No,785.655883,38463.495880


In [9]:
stu=(defa["student"]=="Yes").astype(int)
x1=defa["balance"].values.reshape(-1,1)
x3=stu.values.reshape(-1,1)
x2=defa["income"].values.reshape(-1,1)
y=defa["default"]== "Yes"
# Matriz de predictores
X3=np.hstack([x1,x2,x3])

modelo=LogisticRegression()
modelo.fit(X3,y)

#Intercepto y coeficientes
beta0_3=modelo.intercept_
beta1_3=modelo.coef_

print("Intercept: ",beta0_3)
print("Coeficientes: ",beta1_3)

defa['student_Yes'] = pd.get_dummies(defa['student'], drop_first=True)
X3 = defa[['balance', 'income', 'student_Yes']]
y= defa['default']=='Yes'
X_multi = LogisticRegression()
X_multi.fit(X3, y)

y_pred = modelo.predict(X3)
acc1 = accuracy_score(y, y_pred)
print("Accuracy del modelo (versión 1):", acc1)

Intercept:  [-10.90178652]
Coeficientes:  [[ 5.73059385e-03  3.96180616e-06 -6.12575695e-01]]
Accuracy del modelo (versión 1): 0.9732


C:\Users\munoz\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


Aunque el modelo presenta un accuracy de 97.32%, este resultado debe interpretarse con precaución debido al desbalance del conjunto de datos, ya que la mayoría de los casos corresponden a “No default”

In [12]:
# Convertir X3 a float
X3 = X3.astype(float)
y = (defa['default'] == 'Yes').astype(int)

n_boot = 1000
coef_samples = np.zeros((n_boot, X3.shape[1] + 1))  # +1 para intercepto

for i in range(n_boot):
    sample_indices = np.random.choice(range(len(y)), size=len(y), replace=True)
    X_sample = X3.iloc[sample_indices]
    y_sample = y.iloc[sample_indices]

    # statsmodels requiere float explícito
    X_sample_const = sm.add_constant(X_sample).astype(float)

    logit = sm.Logit(y_sample, X_sample_const)
    resultados = logit.fit(disp=0)
    coef_samples[i, :] = resultados.params

coef_means = np.mean(coef_samples, axis=0)
coef_stds = np.std(coef_samples, axis=0)

print("Coeficientes medios:", coef_means)
print("Desviación estándar:", coef_stds)

Coeficientes medios: [-1.08853113e+01  5.75125827e-03  2.74864131e-06 -6.55026077e-01]
Desviación estándar: [4.93404901e-01 2.34362162e-04 8.49884914e-06 2.45275192e-01]


El intercepto y los coeficientes principales mantienen valores consistentes, con desviaciones estándar bajas. El primer predictor tiene un efecto pequeño pero muy estable, el segundo prácticamente no influye en el resultado, y el coeficiente negativo reduce moderadamente la probabilidad del evento. En conjunto, las bajas variaciones indican que el modelo es robusto y que algunas variables tienen poca relevancia en la predicción.

In [63]:
n_bootstraps = 500
all_cols = X3.columns
pred_samples = np.zeros((n_bootstraps, len(y)))
selected_cols_list = []
models = []
for i in range(n_bootstraps):

    selected_cols = np.random.choice(all_cols, size=2, replace=False)
    selected_cols_list.append(selected_cols)
    idx = np.random.choice(len(y), size=len(y), replace=True)
    X_sample = X3.iloc[idx][selected_cols]
    y_sample = y[idx]

    model = DecisionTreeClassifier()
    model.fit(X_sample, y_sample)
    models.append(model)

for i in range(n_bootstraps):
    selected_cols = selected_cols_list[i]
    pred_samples[i, :] = models[i].predict_proba(X3[selected_cols])[:, 1]

bagged_pred = pred_samples.mean(axis=0)
auc_bagging = roc_auc_score(y, bagged_pred)

print("AUC del Bagging:", auc_bagging)

AUC del Bagging: 1.0


Un accuracy de 1.0 indica que el modelo no cometió ningún error de clasificación en las observaciones evaluadas. Sin embargo, en árboles de decisión esto suele significar que hay oferfiting, porque al no delimitar hasta donde queremos que llegue nuestro árbol, siempre se ira hasta la última hoja, la cual, va a dar 1.